This notebook is intended to demonstrate how select registration, segmentation, and image mathematical methods of ITKTubeTK can be combined to perform multi-channel brain extraction (aka. skull stripping for patient data containing multiple MRI sequences).

There are many other (probably more effective) brain extraction methods available as open-source software such as BET and BET2 in the FSL package (albeit such methods are only for single channel data).   If you need to perform brain extraction for a large collection of scans that do not contain major pathologies, please use one of those packages.   This notebook is meant to show off the capabilities of specific ITKTubeTK methods, not to demonstration how to "solve" brain extraction.

In [1]:
import itk
from itk import TubeTK as tube

from itkwidgets import view

import numpy as np
import time

In [2]:
ImageType = itk.Image[itk.F, 3]

InputBaseName = "../data/CT/CT"
OutputBaseName = "results/CT"

In [3]:
im1 = itk.imread(InputBaseName + ".nii", itk.F)

In [4]:
resamp = tube.ResampleImage[ImageType].New(Input = im1)
resamp.SetMakeIsotropic(True)
resamp.Update()
im1Iso = resamp.GetOutput()

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


In [5]:
itk.imwrite(im1Iso, OutputBaseName + "-Iso.mha")

In [6]:
ImageMath = tube.ImageMath.New(im1Iso)
ImageMath.ConnectedComponents(-1024,-900,1,0,0,0)
ImageMath.Dilate(10,1,0)
ImageMath.Erode(20,1,0)
ImageMath.Dilate(11,1,0)
im1BkgMask = ImageMath.GetOutput()

In [7]:
itk.imwrite(im1BkgMask, OutputBaseName+'-BkgMask.mha')

In [17]:
ImageMath.SetInput(im1Iso)
ImageMath.ReplaceValuesOutsideMaskRange(im1BkgMask,0,0,-1024)
ImageMath.Threshold(-1023,-500,1,0)
ImageMath.Dilate(5,1,0)
ImageMath.Erode(10,1,0)
ImageMath.Dilate(3,1,0)
im1LungPlusMask = ImageMath.GetOutputUChar()

ConnComp = tube.SegmentConnectedComponents.New(im1LungPlusMask)
ConnComp.SetMinimumVolume(10000)
ConnComp.Update()
im1LungMask = ConnComp.GetOutput()

In [18]:
itk.imwrite(im1LungMask, OutputBaseName+'-LungMask.mha')

In [19]:
im1LungMask = itk.imread(OutputBaseName+'-LungMask.mha', itk.F) # Read into float array
ImageMath.SetInput(im1Iso)
ImageMath.ReplaceValuesOutsideMaskRange(im1LungMask,1,255,-1024)
im1Lung = ImageMath.GetOutput()

In [20]:
itk.imwrite(im1Lung, OutputBaseName+'-Lungs.mha')